## Set the path of where the csv files are located, as well as initial import statements

In [11]:
csv_data_path = "/home/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_ocellID_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = True # use smaller ML model for better performance, less accuracy

In [12]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Functions to handle data loading and data writing

In [13]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [14]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords.append(['so', 'just', 'maybe'])
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [15]:
data = load_data(csv_data_path)['content']
data.head()

0    Welcome to our very new community forum.  | “W...
1    Option1:   Sign up for an account at https://O...
2    Hi, I checked around, and I can’t seem to find...
3    Hi, First of all, thank you for providing such...
4    I came across the Mozilla Location Service her...
Name: content, dtype: object

# Content Preprocessing

In [16]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

'welcome ve'

In [17]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
tokenized_data[0][:10]

['welcome',
 'very',
 'new',
 'community',
 'forum',
 'good',
 'way',
 'begin',
 'say',
 'introduce']

# Generate id2word dictionary

In [18]:
id2word = corpora.Dictionary(tokenized_data)

corpus = []

for text in tokenized_data:
    new = id2word.doc2bow(text)
    corpus.append(new)

corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1)]

# LDA Visualization

In [28]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=5,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.043156  0.126506       1        1  36.299298
1     -0.037906 -0.121311       2        1  18.424387
0     -0.177326 -0.006198       3        1  17.333219
2      0.119953 -0.085877       4        1  16.761611
3      0.138436  0.086881       5        1  11.181484, topic_info=         Term       Freq       Total Category  logprob  loglift
93   location  52.000000   52.000000  Default  30.0000  30.0000
332       map  22.000000   22.000000  Default  29.0000  29.0000
178     tower  79.000000   79.000000  Default  28.0000  28.0000
341    upload  15.000000   15.000000  Default  27.0000  27.0000
426      year  11.000000   11.000000  Default  26.0000  26.0000
..        ...        ...         ...      ...      ...      ...
8        here   3.282275   20.227149   Topic5  -5.1652   0.3724
65       know   3.712212   35.858503   Topic5  -5.0421  -0.0770
83         so   3.492496   36.773095   Topic5  -5.1031  -0.1632
76       cell   3.523808  103.803171   Topic5  -5.0942  -1.1920
72      thank   3.116494   49.004059   Topic5  -5.2170  -0.5643

[343 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
156       1  0.312864   access
156       2  0.078216   access
156       3  0.547512   access
531       1  0.292760   accord
531       5  0.585521   accord
...     ...       ...      ...
588       4  0.829161    world
426       4  0.953952     year
926       2  0.790430      yet
1107      1  0.841005     zero
1165      3  0.677585  телефон

[510 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])